# Outlier Detection

### Load necessary package

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns

rdgn = sns.diverging_palette(240, 10, n = 9, as_cmap=True)


# Get the current working directory (assuming the notebook is in the same directory as the file)
current_directory = os.getcwd()

# Get the parent directory
parent_directory = os.path.dirname(current_directory)

import sys
sys.path.append(parent_directory)
from py_scripts.utils import data_library

### load data

In [2]:
dls = data_library()
univ = dls.fetch_univ("SP500_Clean")
err, df = dls.fetch_pv(univ)

### detect price outlier

In [5]:
# as we are mainly interest in the Close price, we can add some sanity check
# 1. Close price should be (1) positive (2) within high and low (3) the daily return should be in a reasonable range

# 1. Close price should be positive
assert df[df['Close'] > 0].shape[0] == df.shape[0]

# 2. Close price should be within high and low
assert df[(df['Close'] >= df['Low']) & (df['Close'] <= df['High'])].shape[0] == df.shape[0]

In [4]:
low_ind = np.where(df['Close'] < df['Low'])[0]
df.iloc[low_ind].head().to_latex(index=False, caption='Example of the Close price is lower than the Low price', label='tab:low_close')

'\\begin{table}\n\\caption{Example of the Close price is lower than the Low price}\n\\label{tab:low_close}\n\\begin{tabular}{lrrrrrrl}\n\\toprule\nDate & Open & High & Low & Close & Volume & OpenInt & Symbol \\\\\n\\midrule\n2009-09-01 & 27.654000 & 27.760000 & 27.201000 & 2.727200 & 3143106 & 0 & eix \\\\\n2012-03-08 & 36.682000 & 36.887000 & 36.347000 & 3.636600 & 2189498 & 0 & eix \\\\\n2012-03-22 & 36.029000 & 36.424000 & 35.936000 & 3.636600 & 2202028 & 0 & eix \\\\\n1994-02-11 & 9.130000 & 9.189400 & 9.036100 & 0.911200 & 706091 & 0 & de \\\\\n2007-01-16 & 45.350000 & 45.383000 & 44.723000 & 4.495600 & 3508852 & 0 & nue \\\\\n\\bottomrule\n\\end{tabular}\n\\end{table}\n'

In [10]:
prc_cols = ['Open', 'High', 'Low', 'Close']

df['ref_price'] = df[prc_cols].apply(lambda x: np.median(x), axis=1)

# need to run the code twice
for col in ['Open', 'High', 'Low', 'Close']:
    err_ind = np.where((df[col] < df['ref_price']) & (df[col] < df['ref_price']*0.2))[0]
    # print(col)
    # print(df.loc[err_ind])
    df.loc[err_ind, col] *= 10
    print(f"fix {col} column with {len(err_ind)} rows")

fix Open column with 2 rows
fix High column with 0 rows
fix Low column with 2 rows
fix Close column with 2 rows


In [13]:
### test again - pass
assert df[(df['Close'] >= df['Low']) & (df['Close'] <= df['High'])].shape[0] == df.shape[0]

In [16]:
open_ind = np.where(np.abs(df['Close']/df['Open'] - 1)>0.8)[0]
df.iloc[open_ind]

Date    Open    High     Low   Close   Volume  OpenInt Symbol  \
891657  2008-09-16  34.688  98.249  23.437  70.313  3204488        0    aig   

        ref_price  
891657    52.5005

In [21]:
df['Date'] = pd.to_datetime(df['Date'])

In [24]:
df = df[df['Date']>=pd.to_datetime('2005-01-01')].reset_index(drop = True)

In [25]:
df.to_parquet("../Data/Panel/SP500_Clean.pq")